# fuzzy yona

In [125]:
import pandas as pd
from fuzzywuzzy import fuzz

pd.set_option('display.max_columns', None)  # Esto mostrará todas las columnas
pd.set_option('display.max_colwidth', None)  # Esto evitará que el contenido de las celdas se trunque

In [126]:
scrapeo_dia = pd.read_csv("../data/scrapeo(2023-07-10 14-50-32).csv", index_col = 0)
datamarket_dia = pd.read_csv("../data/datamarket-dia.csv", index_col = 0)

In [127]:
cat_nuevas = scrapeo_dia['category'].unique().tolist()
sub_nuevas = scrapeo_dia['subcategory'].unique().tolist()
cat_antiguas = datamarket_dia['category'].unique().tolist()

In [142]:
def cat_max(col, limpita):

    maximo = 0

    for l in limpita:

        parecido = fuzz.ratio(col, l)

        if parecido > maximo:

            maximo = parecido
            
            categoria = l

    return categoria

spliteada = []

for s in cat_nuevas:

    a = s.split('_')

    spliteada.append(a)


flat_list = []

for sublist in spliteada:
    for item in sublist:
        flat_list.append(item)


limpita = []

for f in flat_list:
    if f == 'y' or f == 'e' or f == 'con':
        pass
    else:
        limpita.append(f)


apendeos = ['gluten', 'lacteos', 'navidad', 'dieteticos', 'solidario', 'aceitunas', 'sal', 
            'cuidado', 'internacional', 'mermeladas', 'licores', 'sopas', 'espumosos']

for a in apendeos:
    
    limpita.append(a)



aux =[]

for c in cat_antiguas:

    aux.append(c.split('_'))


aux2 = []

word = ''

descartes = ['y', 'e', 'vinagre', 'con', 'del', 'al', 'de', 'dia', 'alimentacion', 'fresco', 'frescos', 
            'desayuno', 'despensa', 'bodega', 'drogueria', 'desayunos', 'dulces',
            'cocina', 'personal', 'soy', 'bano', 'corporal', 'preparacion', 'bebidas']

for a in aux:

    for i in a:
        
        if i in descartes:
            pass

        else:
            word += i + ' '
    
    aux2.append(word.rstrip(' '))

    word = ''

aux3 = []

for a in aux2:

    b = a.split(' ')
    aux3.append(b[0])


df = pd.DataFrame()
df['antiguas'] = pd.DataFrame(aux3)

df['fuzzcat'] = df.apply(lambda x: cat_max(x['antiguas'], limpita), axis = 1)

definitiva = []
aux = []
for sublist in spliteada:
    for f in sublist:
        if f == 'y' or f == 'e' or f == 'con':
            pass
        else:
            aux.append(f)
    definitiva.append(aux)
    aux =[]
definitiva

def convertir_lista_a_diccionario(lista, claves):
    dictio = {}
    
    for i, sublist in enumerate(lista):
        clave = claves[i]  # Generamos las claves como 'key1', 'key2', etc.
        dictio[clave] = sublist
    
    return dictio

lista_de_listas = definitiva
dictio = convertir_lista_a_diccionario(lista_de_listas, cat_nuevas)

dictio['gluten'] = ['gluten'] # sorry celiacos
dictio['leche_huevos_y_mantequilla'].append('lacteos')
dictio['navidad'] = ['navidad']
dictio['galletas_bollos_y_cereales'].append('dieteticos')
dictio['solidario'] = ['solidario']
dictio['patatas_fritas_encurtidos_y_frutos_secos'].append('aceitunas')
dictio['aceites_salsas_y_especias'].append('sal')
dictio['perfumeria_higiene_salud'].append('cuidado')
dictio['internacional'] = ['internacional']
dictio['azucar_chocolates_y_caramelos'].append('mermeladas')
dictio['cervezas_vinos_y_bebidas_con_alcohol'].append('licores')
dictio['conservas_caldos_y_cremas'].append('sopas')
dictio['cervezas_vinos_y_bebidas_con_alcohol'].append('espumosos')

listiña = []

for i in df['fuzzcat']:

    for key, values in dictio.items():
        if i in values:
            listiña.append(key)


df['feas'] = cat_antiguas
df['transformadas'] = listiña

df.sample(20)

,antiguas,fuzzcat,feas,transformadas
423,limpieza,limpieza,drogueria_y_limpieza_cuidado_ropa__detergente_maquina_tabletas,limpieza_y_hogar
506,alcoholes,alcohol,bebidas_alcoholes_y_licores_premium,cervezas_vinos_y_bebidas_con_alcohol
733,pescado,pescados,frescos_pescado_y_marisco_gulas_y_surimi,pescados_mariscos_y_ahumados
558,carne,carniceria,al_dia_carne_mixto,carniceria
670,cuidado,cuidado,cuidado_personal_afeitado,perfumeria_higiene_salud
766,carne,carniceria,frescos_carne_otras_aves,carniceria
99,caldos,caldos,alimentacion_caldos_sopas_y_pures_sopas_en_sobre,conservas_caldos_y_cremas
581,bolleria,bollos,despensa_desayunos_y_dulces_bolleria,galletas_bollos_y_cereales
74,verdura,verduras,frescos_verdura_y_hortalizas_lechuga_y_endibias,verduras
160,refrescos,refrescos,bebidas_refrescos_polvos_para_preparar,agua_refrescos_y_zumos
